In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import numpy as np

# gpus = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
def describe_distributions(distributions):
    print('\n'.join([str(d) for d in distributions]))

In [4]:
poisson_distributions = [
	tfd.Poisson(rate=1., name='One Poisson Scalar Batch'),
	tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons'),
	tfd.Poisson(rate=[[1., 10., 100.,], [2., 20., 200.]],name='Two-by-Three Poissons'),
	tfd.Poisson(rate=[1.], name='One Poisson Vector Batch'),
	tfd.Poisson(rate=[[1.]], name='One Poisson Expanded Batch')
	]

2022-05-03 22:38:01.954700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 22:38:01.955289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-05-03 22:38:01.955507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-05-03 22:38:01.955715: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_

In [5]:
describe_distributions(poisson_distributions)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Expanded_Batch", batch_shape=[1, 1], event_shape=[], dtype=float32)


# Sampling Scalar Distributions

In [6]:
normal_distributions = [
	tfd.Normal(loc=0., scale=1., name='Standard'),
	tfd.Normal(loc=[0.], scale=1., name='Standard Vector Batch'),
	tfd.Normal(loc=[0., 1., 2., 3.], scale=1., name='Different Locs'),
	tfd.Normal(loc=[0., 1., 2., 3.], scale=[[1.], [5.]],  #自动识别有８个组合，所以是２行４列
	name='Broadcasting Scale'),
    # tfd.Normal(loc=[0., 1., 2., 3.], scale=[1., 5.],
	# name='Broadcasting Scale'),  # ValueError: Arguments `loc` and `scale` must have compatible shapes; loc.shape=(4,), scale.shape=(2,).
    tfd.Normal(loc=[0., 1., 2., 3.], scale=[1., 5., 0, 0],
	name='Broadcasting Scale'),
    tfd.Normal(loc=[[1.], [5.]], scale=[0., 1., 2., 3.],
	name='Broadcasting Scale'),  #自动识别有８个组合，所以是２行４列
	]

In [7]:
describe_distributions(normal_distributions)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
tfp.distributions.Normal("Broadcasting_Scale", batch_shape=[2, 4], event_shape=[], dtype=float32)
tfp.distributions.Normal("Broadcasting_Scale", batch_shape=[4], event_shape=[], dtype=float32)
tfp.distributions.Normal("Broadcasting_Scale", batch_shape=[2, 4], event_shape=[], dtype=float32)


In [8]:
def describe_sample_tensor_shape(sample_shape, distribution):
    print('Sample shape:', sample_shape)
    print('Returned sample tensor shape:', distribution.sample(sample_shape).shape)

def describe_sample_tensor_shapes(distributions, sample_shapes):
    started = False
    for distribution in distributions:
        print(distribution)
        for sample_shape in sample_shapes:
            describe_sample_tensor_shape(sample_shape, distribution)
        print()

sample_shapes = [1, 2, [1, 5], [3, 4, 5]]  # ４次sample
describe_sample_tensor_shapes(poisson_distributions, sample_shapes)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

In [9]:
describe_sample_tensor_shapes(normal_distributions, sample_shapes)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 1)
Sample shape: 2
Returned sample tensor shape: (2, 1)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 1)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 1)

tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 4)
Sample shape: 2
Returned sample tensor shape: (2, 4)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 4)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 4)

tfp.distributions.Normal("Broadc

# Computing log_prob For Scalar Distributions

In [10]:
three_poissons = tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons')
three_poissons

<tfp.distributions.Poisson 'Three_Poissons' batch_shape=[3] event_shape=[] dtype=float32>

In [11]:
three_poissons.log_prob([[1., 10., 100.], [100., 10., 1]]) # sample_shape is [2]

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -2.0785608,   -3.2223587],
       [-364.73938  ,   -2.0785608,  -95.39484  ]], dtype=float32)>

In [12]:
sample_tensor = [[[[1., 10., 100.], [100., 10., 1.]]]]
display(tf.constant(sample_tensor).shape)
print()
print('Sample tensor的shape是[1, 1, 2, 3]')
print('Distribution `three_poissons`的batch_shape是[3], event_shape是[]')
print('因此，根据sample tensor shape=(*sample_shape, *batch_shape, *event_shape)，TensorFlow可以自动infer得知上面的sample tensor的sample_shape是[1, 1, 2]')
print()
three_poissons.log_prob(sample_tensor)

TensorShape([1, 1, 2, 3])


Sample tensor的shape是[1, 1, 2, 3]
Distribution `three_poissons`的batch_shape是[3], event_shape是[]
因此，根据sample tensor shape=(*sample_shape, *batch_shape, *event_shape)，TensorFlow可以自动infer得知上面的sample tensor的sample_shape是[1, 1, 2]



<tf.Tensor: shape=(1, 1, 2, 3), dtype=float32, numpy=
array([[[[  -1.       ,   -2.0785608,   -3.2223587],
         [-364.73938  ,   -2.0785608,  -95.39484  ]]]], dtype=float32)>

In [13]:
three_poissons.log_prob([10.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-16.104412 ,  -2.0785608, -69.05272  ], dtype=float32)>

In [15]:
sample_tensor = [[[1.], [10.]], [[100.], [1000.]]]
display(np.shape(sample_tensor))

three_poissons.log_prob(sample_tensor)

(2, 2, 1)

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[-1.0000000e+00, -7.6974149e+00, -9.5394836e+01],
        [-1.6104412e+01, -2.0785608e+00, -6.9052719e+01]],

       [[-3.6473938e+02, -1.4348087e+02, -3.2223587e+00],
        [-5.9131279e+03, -3.6195427e+03, -1.4069575e+03]]], dtype=float32)>

test value

In [38]:
tfd.Poisson(rate=[200.]).log_prob(100.)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-33.90767], dtype=float32)>

In [24]:
poisson_2_by_3 = tfd.Poisson(
    rate=[[1., 10., 100.,], [2., 20., 200.]],
    name='Two-by-Three Poissons')

In [25]:
poisson_2_by_3.log_prob(1.)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [26]:
poisson_2_by_3.log_prob([1.])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [27]:
poisson_2_by_3.log_prob([[1., 1., 1.], [1., 1., 1.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [28]:
poisson_2_by_3.log_prob([[1., 10., 100.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [29]:
poisson_2_by_3.log_prob([[1., 10., 100.], [1., 10., 100.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [30]:
poisson_2_by_3.log_prob([[1., 1., 1.], [2., 2., 2.]]) # input sample tensor's sample_shape == 2, batch_shape == 3

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

In [31]:
poisson_2_by_3.log_prob([[1.], [2.]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

In [32]:
poisson_2_by_3.log_prob([[[1., 1., 1.], [1., 1., 1.]], [[2., 2., 2.], [2., 2., 2.]]])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

In [33]:
poisson_2_by_3.log_prob([[[1.], [1.]], [[2.], [2.]]])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

In [34]:
poisson_2_by_3.log_prob([[[1., 1., 1.]], [[2., 2., 2.]]])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

In [35]:
poisson_2_by_3.log_prob([[[1.]], [[2.]]])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

In [37]:
poisson_2_by_3.log_prob(tf.constant([1., 2.])[..., tf.newaxis, tf.newaxis])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

In [38]:
tf.constant([1., 2.])[..., tf.newaxis, tf.newaxis]

<tf.Tensor: shape=(2, 1, 1), dtype=float32, numpy=
array([[[1.]],

       [[2.]]], dtype=float32)>

In [39]:
tf.constant([1., 2.])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [40]:
three_poissons.log_prob([[1.], [10.], [50.], [100.]])

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

In [41]:
three_poissons.log_prob(tf.constant([1., 10., 50., 100.])[..., tf.newaxis])

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

In [42]:
three_poissons.log_prob(tf.expand_dims(tf.constant([1., 10., 50., 100.]),axis=-1))

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

# Multivariate distributions

In [58]:
a = tfd.Multinomial(total_count=100., probs=[.5, .4, .1],
    name='One Multinomial')
a

<tfp.distributions.Multinomial 'One_Multinomial' batch_shape=[] event_shape=[3] dtype=float32>

In [64]:
sample_tensor_shape = 1
sample_tensor = a.sample(sample_tensor_shape)
sample_tensor

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[48., 45.,  7.]], dtype=float32)>

In [65]:
a.prob(sample_tensor)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00548431], dtype=float32)>

In [66]:
sample_tensor_shape = 3
sample_tensor = a.sample(sample_tensor_shape)
sample_tensor

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[40., 50., 10.],
       [46., 42., 12.],
       [58., 34.,  8.]], dtype=float32)>

In [67]:
a.prob(sample_tensor)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.0011949 , 0.00692727, 0.00341546], dtype=float32)>

In [49]:
multinomial_distributions = [
# Multinomial is a vector-valued distribution: if we have k classes,
# an individual sample from the distribution has k values in it, so the
# event_shape is `[k]`.
    tfd.Multinomial(total_count=100., probs=[.5, .4, .1], name='One Multinomial'),
    tfd.Multinomial(total_count=[100., 1000.], probs=[.5, .4, .1], name='Two Multinomials Same Probs'),
    tfd.Multinomial(total_count=100., probs=[[.5, .4, .1], [.1, .2, .7]], name='Two Multinomials Same Counts'),
    tfd.Multinomial(total_count=[100., 1000.], probs=[[.5, .4, .1], [.1, .2, .7]],
    name='Two Multinomials Different Everything')
]

In [50]:
describe_distributions(multinomial_distributions)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Different_Everything", batch_shape=[2], event_shape=[3], dtype=float32)


In [51]:
sample_shapes

[1, 2, [1, 5], [3, 4, 5]]

In [52]:
describe_sample_tensor_shapes(multinomial_distributions, sample_shapes)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]


In [68]:
two_multivariate_normals = tfd.MultivariateNormalDiag(loc=[1., 2., 3.], scale_identity_multiplier=[1., 2.])
two_multivariate_normals

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag' batch_shape=[2] event_shape=[3] dtype=float32>

In [79]:
sample_tensor = two_multivariate_normals.sample(1) # sample_shape==1, batch_shape==2, event_shape==3
sample_tensor

<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[-0.16367447,  1.723098  ,  3.5098622 ],
        [ 2.1409044 ,  3.6139274 ,  1.8398367 ]]], dtype=float32)>

In [80]:
two_multivariate_normals.prob(sample_tensor)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.02726363, 0.00411627]], dtype=float32)>

In [73]:
two_multivariate_normals.log_prob([[[1., 2., 3.]], [[3., 4., 5.]]]) # Input has shape [2,1,3].

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

In [75]:
two_multivariate_normals.log_prob(
    tf.constant([[1., 2., 3.], [3., 4., 5.]])[:, tf.newaxis, :]) # Equivalent to above.

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

In [76]:
two_multivariate_normals.log_prob(tf.constant([[1., 2., 3.], [3., 4., 5.]]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.7568154, -6.336257 ], dtype=float32)>

# Shape Manipulation Techniques